# DAY 13: Nonnegative Matrix Factorization


### Machine Learning and Computational Statistics (DSC6232)

#### Instructors: Weiwei Pan, Melanie Fernandez, Pavlos Protopapas

#### Due: August 12th, 2:00 pm Kigali Time

**First name**: LONGIN


**Last name**: DUSENGEYEZU

**ID**:220020609

## Learning Goals:

1. learn how to process and encode text data
2. understand how to analyze documents using a simple topic model 
3. learn how to interpret nonnegative matrix factorization models

### Load necessary libraries

In [ ]:
import random
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from IPython.display import display
%matplotlib inline


### We include auxiliary functions here that we will need to use later  **No need to read in details!**

We include auxiliary functions here that we will need to use later



In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

def print_features(feature_names, num_columns=5):
    padding = num_columns - len(feature_names) % num_columns
    feature_names += [''] * (padding * (padding != num_columns))
    feature_names = np.array(feature_names).reshape(-1, num_columns)
    display(pd.DataFrame(feature_names, columns=[''] * num_columns).reset_index(drop=True))

# Topic Modeling for News Articles

This exercise is designed to help you transform and model textual data. You may find the tutorial [here](http://scikit-learn.org/stable/modules/feature_extraction.html) helpful.

You will encode a small set of news articles (i.e. represent them as count vectors) and model this set using a Nonnegative Factorization Model. Your goal is to discover a latent set of topic underlying the articles and discover which topics appear in each article.


### Load-in the data and examine it

We use the `fetch_20newsgroups` function from `sklearn` to load a set of articles in the categories: "medicine", "religion" and "motorcycles".

In [ ]:
# Load the data set
print("Loading dataset...")
data, _ = fetch_20newsgroups(shuffle=False, remove=('headers', 'footers', 'quotes'),
                             return_X_y=True, categories=['sci.med', 'soc.religion.christian', 'rec.motorcycles'])
print('Done.')

Loading dataset...
Done.


We check to see how many articles we have loaded. We also print two articles form this set to see what they look like.

In [ ]:
# Print the number of articles in the data
print('Number of data points: {}\n'.format(len(data)))

# Print out an example article from the data
print('Example articles:\n\n')
print('*' * 10 + ' Example 1 ' + '*' * 10)
print(data[0])

# Print out another article from the data
print('\n\n' + '*' * 10 + ' Example 2 ' + '*' * 10)
print(data[5])

Number of data points: 1791

Example articles:


********** Example 1 **********
Does anyone on this newsgroup happen to know WHY morphine was
first isolated from opium?  If you know why, or have an idea for where I
could look to find this info, please mail me.
	CSH
any suggestionas would be greatly appreciated

--
 "Kilimanjaro is a pretty tricky climb. Most of it's up, until you reach
the very, very top, and then it tends to slope away rather sharply."
					Sir George Head, OBE (JC)


********** Example 2 **********
I just noticed that my halogen table lamp runs off 12 Volts.
The big thinngy that plugs into the wall says 12 Volts DC,  20mA

The question is: Can I trickle charge the battery on my CB650
with it?

I don't know the rating of the battery, but it is a factory
intalled one. 


Thanks,
Sanjay

-- 
   '81 CB650 						DoD #1224


### Encode the data as count vectors

We are going to use `sklearn`'s `TfidfVectorizer` function to remove punctuations and non-meaningful words from the documents and then convert them into count vectors.

**Exercise 1:** Read the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) for `TfidfVectorizer`, and experiment with different values for the parameters `max_df`, `min_df`, `max_features`. What do each of these parameters mean? How does changing these parameters change the count vector representation of the data?

**Answer**:

**TfindVectorizer: Convert a collection of raw documents to a matrix of TF-IDF features.**

**1) max_df : stands for maximum document frequency**.When building the vocabulary, ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float in range [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.


Here if frequence of word in a document exceeds threshold specified now is going to be removed in the list of words that could help us to identify to pic, because may be is the word that is frequently used but it has not meaning that could help us to identify natural of topic.

**2)min_df: stands for minimum  document frequency:** When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float in range of [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

Here if frequence of word in a document is below  threshold specified now is going to be removed in the list of words that could help us to identify to pic, because maybe is the word that is not  frequently used ,it has not have enough information   that could help us to identify natural of topic.

**2)max_features:**
If not None, build a vocabulary that only considers the top max_features ordered by term frequency across the corpus.
This really help us to set all words in matrix form just to represent all words that form each topic 



**NB: from this experiment and from the definition of each parameter we have seen that if we increase max_df, the number of words in matrix or per topic is increased but decrease if max_df is reduced. From min_df is increased number of word is reduced but contrally if increased**  


In [ ]:
# Step 1: Reduce the size of the data
n_samples = 1000
data_samples = random.sample(data, min(n_samples, len(data)))

# Step 2: Choose the number of features, or important words, to extract
n_features = 1000

# Step 3: Extract tf-idf features
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')

# Step 4: Encode the documents as normalized count vectors
vectorized_data = tfidf_vectorizer.fit_transform(data_samples)

# Step 5: Get learned feature names
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# Step 6: Select a samplee of the learned features 
sample_of_features = random.sample(tfidf_feature_names, 100)

# Step 7: Print that sample of learned feature names
print_features(sample_of_features)

,,,,,
0,send,woman,following,nature,animals
1,17,newsletter,deal,able,mean
2,letter,specific,completely,disciples,claims
3,news,national,forget,methods,treated
4,certainty,medical,exactly,realize,prayers
5,road,pitt,sins,effects,universe
6,outside,directly,major,friend,divine
7,effect,changed,wish,period,posting
8,doesn,power,modern,19,knowledge
9,spiritual,trouble,jewish,careful,say


**Exercise 2:** Print the normalized count vector representation of a single document. What kind of numbers are in this vector? What do these numbers represent? ***Hint:*** recall how we process count vectors before fitting a nonnegative matrix factorization model. 

**Answer**:

The numbers on the left are one-hot encoded identifiers of the different words. On the right, these numbers are frequencies of occurrence of specific numbers in the documents

The `TfidfVectorizer` function normalizes the count vectors, what does this mean and why is this step necessary?


**Answer**:

This means that the function expresses the word counts as percentage frequencies. This step is necessary in extracting the most important features or words in the documents. Also this percentage is needed in needed in matrix represtation even in NMF model in dowun word in this notebook 

In [ ]:
# Step 1: Print the normalized count vector representation of a single document
n = 10
print('The normalized ount vector representation of the {}-th document'.format(n))
print(vectorized_data[n])

The normalized ount vector representation of the 10-th document
  (0, 696)	0.3536207863458619
  (0, 611)	0.507835027091325
  (0, 961)	0.3811902989395354
  (0, 263)	0.3307424291717216
  (0, 641)	0.3307424291717216
  (0, 519)	0.24510009791277151
  (0, 806)	0.27453259784821016
  (0, 826)	0.3021021104418836
  (0, 513)	0.16205209470190518


### Fit a Nonnegative Matrix Factorization Model to the data

Now that our data has been encoded as normalized count vectors, we can fit an NMF model to it.

**Exercise 3:** Fit an NMF model with 10 topics, print out the top words associated to each topic. Can you interpret what each topic is about?

**Answer**:

Yes. I can interpret what most topics are about. For example, topic 2 mentions biblical names and words such as Jesus and sin, signifying christian preaching(religion). Topic 3 is a scientific theory and process(motorcycles).Topic 4 is about food and nutrition(medicine). Topic 5 is about health and medicine(medicine). Topic 6 is about catholic church beliefs(religion). Topic 8 is on motorcycles and their insurance(motorcycles)

NB: even if we can identify same but other we can not and i think this is not good number of topic we can choose.

Fit an NMF model with 2 topics, print out the top words associated to each topic. Can you interpret what each topic is about?

**Answer**:

 
It is hard to interpret what each topic is about when we are limited to 2 topics. and it became completly worse compered to the result where we have used 10 topics.


Find an appropriate number of topics. Why is this number appropriate?

**Answer**:
The appropriate number of topics would be between 2 and 10. This is because, it ensures that unique words are spread across different topics. This is the factor that makes unique topic identification from the bag of words possible  also alleast you can identify what each topic is all about. but also not 100%.
Here it is better to use same technical method of choosing hyperparameters like cross-validation  or others.



In [ ]:
# Step 1: Fit NMF model
nmf = NMF(n_components=8, alpha=0.1, l1_ratio=0.5).fit(vectorized_data)

# Step 2: Print out the learned topics
print('Topics learned by the NMF:')
print_top_words(nmf, tfidf_feature_names, 10)

Topics learned by the NMF:
Topic #0: don know like just people think time does good ve
Topic #1: cadre shameful dsl geb chastity n3jxp surrender pitt intellect banks
Topic #2: bike helmet bikes ride riding miles honda dod used buy
Topic #3: msg food chinese glutamate eat foods taste people use reaction
Topic #4: god faith bible lord christ hell truth life heaven christians
Topic #5: thanks mail know reference new program hi thing responses different
Topic #6: jesus father christ john son apostles sin come did kingdom
Topic #7: dog dogs road street got saw face running left lane



**Exercise 4:** Pick a document and print out the combinations of topics in that document. Which combinations of topics are contained in this article? Do you agree with the the combination of topics learned by the model?

**Answer**


The document combines the topic of health and motorcycles. I agree with the combination of topics since they both add up to the common theme of healthy living through both eating well and exercising sufficiently  and also even if you can say 100% if about healthy diet and motorcycles but it has higher probability to be those topics.


In [ ]:
# Step 1: get the document to topic matrix each row of this matrix is the combination of topics in the document
document_to_topic = nmf.transform(vectorized_data)

# Step 2: print the shape of this matrix to verify that we have 1000 documents and 10 topic
print(document_to_topic.shape)

(1000, 10)


In [ ]:
# Step 3: print an article and the predicted combination of topics in this article
n = 10

# Print the predicted combination of topics in this article
print('\n\n' + '*' * 10 + ' Predicted combination of topics ' + '*' * 10 + '\n\n')
print(document_to_topic[n])

# Print out an example article from the data
print('\n\n' + '*' * 10 + ' Article {} '.format(n) + '*' * 10)
print(data[n])



********** Predicted combination of topics **********


[0.03705662 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


********** Article 10 **********

I attribute my success to several factors:

Very low fat.  Except when someone else has cooked a meal for me,
I only eat fruit, vegetables, and whole grain or bran cereals.  I
estimate I only get about 5 to 10 percent of my calories from fat.

Very little sugar or salt.

Very high fiber.  Most Americans get about 10 grams.  25 to 35 are
recommended.  I get between 50 and 150.  Sometimes 200.  (I've heard
of people taking fiber pills.  It seems unlikely that pills can
contain enough fiber to make a difference.  It would be about as
likely as someone getting fat by popping fat pills.  Tablets are
just too small, unless you snarf down hundreds of them daily.)

My "clean your plate" conditioning works *for* me.  Eating the last
10% takes half my eating time, and gives satiety a chance to catch
